In [9]:
# from utils import readout_data
import os
import numpy as np

In [11]:
def read_data(path):
    data = {}
    filenames = os.listdir(path)
    sample    = np.load(os.path.join(path, filenames[0]))
    if len(sample.shape) == 2:
        ## read out as one-dimension mapped and normalizated
        for filename in os.listdir(path):
            features = uint8_to_float32(np.mean(np.load(path + '/' + filename), axis=0))
            data[filename[:-4]] = features
    else:
        ## read out as it is
        for filename in os.listdir(path):
            features = np.load(path + '/' + filename)
            data[filename[:-4]] = features   
    return data

def readout_data(audio_path, rgb_path, label_path):
    audio_data, rgb_data, label_data = read_data(audio_path), read_data(rgb_path), read_data(label_path)
    return audio_data, rgb_data, label_data

def uint8_to_float32(x):
    return (np.float32(x) - 128.0) / 256.0

In [13]:
audio_path = '/home/dhzeng/AVIDEO/Data/vgg/'
rgb_path = '/home/dhzeng/AVIDEO/Data/inception/'
label_path = '/home/dhzeng/AVIDEO/Data/VEGAS_classes/'

fold_path = '/home/dhzeng/AVIDEO/Data/folds/'

audio_data, rgb_data, label_data = readout_data(audio_path, rgb_path, label_path)

In [18]:
def train_test_split(data_dict, train_vid, test_vid):
    if not list(data_dict.values())[0].shape:
        ## label
        Train, Test = np.empty((0), np.float32), np.empty((0), np.float32)
    elif len(list(data_dict.values())[0].shape)==2:
        ## when input data dimmension = 2
        data_len =  list(data_dict.values())[0].shape
        Train, Test = np.empty((0, data_len[0], data_len[1]), np.float32), np.empty((0, data_len[0], data_len[1]), np.float32)
    elif len(list(data_dict.values())[0].shape)==1:
        ## when input data dimmension = 1
        data_len =  list(data_dict.values())[0].shape[0]
        Train, Test = np.empty((0,data_len), np.float32), np.empty((0,data_len), np.float32)
    for vid in data_dict:
        data_item = np.expand_dims(data_dict[vid], axis=0)
        if vid in train_vid:
            Train = np.concatenate((Train, data_item), axis=0)
        if vid in test_vid:
             Test = np.concatenate((Test, data_item), axis=0)
    print("Trainning set and Testing shape:", Train.shape, Test.shape)
    return Train.astype('float32'), Test.astype('float32')

def load_fold_ids(fold_path):
    fold_file = open(fold_path)
    return fold_file.read().splitlines()

In [50]:
import itertools

folds_lst = os.listdir(fold_path)
i = 4
train_vid = list(itertools.chain(*[load_fold_ids(os.path.join(fold_path, folds_lst[j])) for j in range(5) if j!=i]))
test_vid = load_fold_ids(os.path.join(fold_path, folds_lst[i]))
# (train_audio, test_audio), (train_rgb, test_rgb), (train_label, test_label) \
#                       = train_test_split(audio_data, train_vid, test_vid),\
#                         train_test_split(rgb_data,   train_vid, test_vid),\
#                         train_test_split(label_data, train_vid, test_vid)

train_label, test_label= train_test_split(label_data, train_vid, test_vid)
test_label[:100]

Trainning set and Testing shape: (22483,) (5620,)


array([7., 1., 9., 2., 8., 9., 6., 6., 6., 4., 2., 8., 6., 1., 1., 1., 6.,
       8., 8., 2., 5., 8., 2., 0., 2., 8., 9., 6., 2., 2., 6., 6., 8., 6.,
       2., 2., 4., 8., 0., 2., 7., 3., 5., 2., 7., 2., 4., 2., 8., 9., 4.,
       2., 7., 7., 4., 1., 6., 8., 4., 8., 7., 9., 6., 0., 5., 6., 7., 3.,
       4., 9., 8., 5., 8., 8., 5., 5., 7., 8., 5., 7., 8., 7., 1., 2., 6.,
       4., 2., 2., 3., 8., 9., 4., 1., 6., 5., 4., 0., 3., 6., 1.],
      dtype=float32)

## TNN_CCA

In [44]:
## tnn-c-cca embeding

audio_path = "/home/dhzeng/TNN_CCCA/embedding/Accca_test_audio_01s.h5.npy"
visual_path = "/home/dhzeng/TNN_CCCA/embedding/Accca_test_visual_01s.h5.npy"
import numpy as np

for i in range(5):
    lab_path = "/home/dhzeng/TNN_CCCA/embedding/test_lab_0"+str(i)+"s.h5.npy"
    lab = np.load(lab_path)
    print(audio_emb.shape, visual_emb.shape, lab.shape, lab[:200])

# audio_emb = np.load(audio_path)
# visual_emb = np.load(visual_path)


(5620, 10) (5620, 10) (5621,) [0. 7. 7. 7. 0. 9. 9. 2. 2. 2. 7. 7. 6. 6. 6. 6. 6. 6. 6. 7. 4. 6. 6. 6.
 8. 8. 4. 2. 2. 0. 0. 6. 6. 0. 0. 0. 0. 0. 0. 0. 0. 2. 1. 1. 1. 1. 5. 2.
 2. 9. 9. 9. 1. 1. 1. 9. 9. 9. 8. 2. 8. 8. 2. 5. 2. 5. 2. 5. 5. 2. 2. 3.
 3. 2. 2. 0. 6. 9. 3. 8. 9. 9. 0. 5. 5. 5. 6. 6. 1. 1. 8. 8. 8. 8. 8. 8.
 4. 4. 4. 4. 4. 4. 1. 1. 1. 2. 4. 8. 8. 4. 5. 5. 5. 0. 1. 1. 1. 1. 8. 8.
 2. 2. 8. 8. 0. 1. 1. 2. 2. 8. 2. 2. 2. 9. 9. 9. 9. 9. 9. 9. 8. 8. 5. 5.
 5. 9. 5. 0. 0. 2. 2. 5. 6. 5. 6. 5. 3. 3. 3. 3. 6. 7. 3. 3. 7. 7. 6. 1.
 1. 1. 4. 4. 6. 1. 1. 6. 6. 6. 6. 6. 6. 6. 6. 8. 6. 5. 5. 5. 5. 8. 2. 2.
 2. 4. 8. 8. 5. 5. 8. 6.]
(5620, 10) (5620, 10) (5620,) [0. 0. 7. 7. 7. 7. 7. 0. 0. 0. 0. 9. 2. 6. 6. 6. 7. 7. 4. 4. 4. 8. 8. 8.
 8. 8. 4. 2. 2. 2. 6. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 2. 1. 1. 1. 1. 1. 1.
 1. 5. 2. 9. 9. 1. 2. 2. 2. 5. 5. 4. 3. 2. 2. 0. 0. 0. 0. 6. 6. 6. 7. 7.
 7. 7. 9. 9. 3. 3. 7. 7. 7. 7. 8. 9. 9. 5. 6. 6. 6. 4. 4. 1. 4. 1. 1. 4.
 5. 4. 6. 8. 8. 8. 4. 4. 4. 4. 4. 5. 5

In [2]:
import _pickle as cPickle

with open('/home/dhzeng/TNN_CCCA/ACMR/data/vegas/test_id_label_map.pkl', 'rb') as f:
    test_labels = cPickle.load(f, encoding='iso-8859-1')

In [25]:
a= np.array(list(test_labels.values()))[:200]

np.where(a==1.0)[1]

array([3, 6, 5, 8, 1, 8, 7, 9, 1, 8, 5, 5, 1, 9, 5, 2, 8, 5, 6, 6, 5, 0,
       0, 6, 1, 2, 6, 5, 8, 8, 9, 7, 4, 1, 0, 2, 0, 8, 1, 4, 4, 4, 4, 9,
       2, 1, 4, 3, 1, 0, 3, 1, 1, 5, 4, 4, 2, 0, 5, 4, 3, 5, 9, 8, 6, 3,
       9, 4, 4, 0, 8, 2, 9, 0, 3, 1, 4, 5, 3, 4, 0, 6, 0, 3, 8, 6, 5, 8,
       4, 5, 4, 4, 2, 0, 4, 8, 9, 8, 4, 2, 1, 8, 3, 9, 5, 4, 2, 1, 8, 2,
       3, 1, 9, 1, 2, 5, 5, 1, 2, 2, 2, 4, 7, 6, 8, 6, 0, 9, 6, 2, 4, 3,
       8, 9, 8, 8, 1, 6, 8, 1, 2, 4, 3, 6, 5, 5, 6, 8, 8, 2, 9, 0, 1, 4,
       8, 1, 4, 6, 9, 7, 2, 2, 5, 8, 2, 1, 6, 6, 3, 7, 3, 8, 3, 1, 8, 0,
       6, 7, 6, 4, 0, 1, 8, 6, 0, 8, 5, 4, 1, 0, 7, 4, 1, 3, 1, 4, 8, 6,
       6, 2])

In [22]:
type(a[0][3])

numpy.float32